# Inference Runbook

## Overview
This notebook provides a step-by-step guide to configuring and submitting building footprint inference requests. Each step corresponds to a heading in the notebook for easier navigation. The 'kernel' for this notebook appears in upper right, should be 'conda_python3'. 
If it is not, please change the kernel via Kernel in file menu.

### Steps
1. **Import Dependencies**
   Load the necessary libraries and modules.

2. **Configure Inference Job**
   Set up the parameters and configurations for the inference job.

3. **AOI / Input GeoTIFF Selection**
   Choose the Area of Interest (AOI) and the input GeoTIFF file.

4. **Start Model Endpoint**
   Initialize the model endpoint for inference.

5. **Run Inference Job**
   Submit the inference request to the model endpoint.

6. **Stop Model Endpoint**
   Shut down the model endpoint after the job completes.

7. **Visualize Results**
   Generate visualizations to review the inference outputs.

8. **Filter GeoJSON Detections**
   Refine the detections by applying filtering criteria.

9. **Save Filtered GeoJSON to Shapefile**
   Save the filtered GeoJSON data as a Shapefile for further use.

# 1. Import Required Dependencies

### Notebook Setup and Configuration

This section sets up the environment and imports the necessary modules to run the inference pipeline.

#### Key Steps:
1. **Enable Auto-Reload**: Automatically reloads Python modules when they are modified using `%load_ext autoreload`.
2. **Path Configuration**: Adds the `inference` directory to the Python path to ensure all required modules can be imported.
3. **Library Imports**: Loads key libraries such as:
   - **Boto3** for AWS service interaction.
   - **SageMaker SDK** for endpoint management.
   - **Custom utilities** from the `utils` module for pipeline-specific functions.
4. **Warning and Logging Management**:
   - Suppresses unnecessary warnings and configures SageMaker logs to show only critical messages.
5. **AWS Environment Initialization**:
   - Sets up a SageMaker session and retrieves essential details, including:
     - Role
     - Region
     - Account ID
   - Prints these details for confirmation.

In [ ]:
# Enable autoreload to automatically reload modules when they are modified
%load_ext autoreload
    
# Add the directory containing our notebook
import sys
module_directory = "/home/ec2-user/guidance-for-processing-overhead-imagery-on-aws/documentation/inference"
sys.path.append(module_directory)

# Import required libraries
from sagemaker import get_execution_role, Session
import boto3
import warnings
import logging
import pprint
from inference_utils import (
    get_input_imagery,
    update_modelrunner_tasks,
    wait_for_endpoint,
    submit_all_images_for_processing,
    wait_for_job_completion,
    merge_and_save_results_local,
    get_geojson_name_for,
    plot_inference_results,
    filter_geojson_with_threshold,
    compare_inference_results,
    save_geojson_as_shapefile,
    create_sagemaker_endpoint,
    delete_sagemaker_endpoint,
    save_config_to_yaml
)
# Initialize PrettyPrinter with specified indentation
pp = pprint.PrettyPrinter(indent=3)

# Suppress specific warnings
warnings.filterwarnings(
    "ignore", 
    message="numpy.dtype size changed"
)

# Set SageMaker logging level to WARNING
logging.getLogger("sagemaker").setLevel(logging.WARNING)

# Optionally, suppress other specific logs if needed
logging.getLogger("sagemaker.config").setLevel(logging.ERROR)

# Initialize SageMaker session and retrieve details
session = boto3.session.Session()
region = session.region_name
sm_boto3 = boto3.client("sagemaker")
sm_session = Session(boto_session=session)
role = get_execution_role()
account = sm_session.account_id()

# Print essential details (limited to avoid clutter)
print(f"Role: {role}")
print(f"Region: {region}")
print(f"Account: {account}")

# 2. Configure Inference Job

### Preparing the Inference Configuration

#### Steps:
1. Update the provided `inference_confg` fields in to match your specific requirements.
2. Run the cell and it will automatically save a copy of your config locally.

#### Important Notes:
- If you edit the configuration file while using this notebook, **make sure to rerun the next 'Load' cell** to ensure the notebook uses the latest version of your configuration.  
- After running the 'Load' cell, verify that your changes are correctly reflected in the printed configuration.

In [ ]:
# Define the inference configuration inline
inference_config = {
    "region": "us-east-1",
    "account": "12345678901",
    "jobName": "inference-test",  # Job name, also used for output prefix/folder.

    # Input Imagery Configuration
    # Uncomment the line below if input type is "aoi" or "years":
    # "imagery_meta_s3_uri": "s3://test-tile-metadata-bucket_test_tile_metadata",
    "imagery_bucket_s3": "test-imagery-bucket",
    "which_input_option": "s3_path",  # Options: 'aoi', 'years', 's3_path'.

    # Uncomment one of these keys based on the input type:
    # "aoi": "s3://nc-building-footprints-app/SurryCounty_aoi.json",
    # "years": ["2022"],  # Example years.
    "s3_path": "s3://osml-test-images-12345678901/small.tif",

    # Model Information
    "model_endpoint_config_name": "aircraft-config",  # Choose your endpoint config.
    "model_endpoint_name": "aircraft",
    "tile_overlap": 512,  # Tile overlap in pixels.

    # Fixed pipeline values
    "modelrunner_cluster_name": "MRCluster",
    "modelrunner_service_name": "OSML-ModelRunner-MRDataplaneMRService79C973F6-Nm6ZwKR4tmgL",
    "modelrunner_task_count": 1,

    # GeoJSON Output Configuration
    "AOI_S3_output_bucket": "mr-bucket-sink-12345678901",
    "AOI_S3_output_prefix": "test/",  # Appends job name automatically.

    # Post-Processing Parameters
    "detection_threshold": 0.3,  # Confidence threshold for detections.
}

save_config_to_yaml(inference_config, "/home/ec2-user/SageMaker/inference/inference_config.yml")

# 3. Input File Selection

### Selecting Files for Inference

The pipeline requires a Python list of GeoTIFFs stored in S3, assigned to the variable `all_images`.

### Input Options:
There are four ways to populate this list:
1. **AOI (Area of Interest)**  
2. **Years**  
3. **S3 Path**  
4. **File**

### Important Notes:
Ensure you specify your desired input method in the configuration file under the `which_input_option` field. If there is a CRS warning from AOI, we can ignore as we're confident the original CRS is indeed EPSG:6543. If there is a CRS warning from AOI, we can ignore as we're confident the original CRS is indeed EPSG:6543

In [ ]:
# Retrieve the list of input imagery based on the configuration
all_images = get_input_imagery(inference_config)

# Verify the number of images retrieved and inspect one of the paths
print(f"Number of images: {len(all_images)}")
if all_images:
    print(f"Example image path: {all_images[0]}")

# 4. Start Model Endpoint

### Starting the AI/ML Model for Inference

Before running the inference job, we need to start the AI/ML model endpoint if it is not already running.
The **final step** of this process will be shutting the model down to avoid unnecessary costs.

### Explore Endpoint Details
- If you're curious, you can check the [SageMaker endpoints in the AWS Console](https://us-east-1.console.aws.amazon.com/sagemaker/home?region=us-east-1#/endpoints) before starting the endpoint, and revisit after it has been started.

### Endpoint Configurations
- The instance type is determined by the `model_endpoint_config` specified in the configuration file.
- To create a new model endpoint configuration for hosting your model on SageMaker, you can use the [AWS SageMaker Create Endpoint Configuration Console](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateEndpointConfig.html).
- [AWS On-Demand Instance Pricing](https://aws.amazon.com/ec2/pricing/on-demand/) provides cost details for each instance type.

### Recommendation:
- Initial benchmarking has been performed using **P3 instances** for optimal performance.
- Once the team is familiar with the platform, switching to **G4 instances** is recommended to reduce costs while still achieving usable inference results.
  
### Inspecting Endpoint Status

It typically takes **5–10 minutes** for the endpoint to become fully operational, as SageMaker provisions the ECS instance hosting it.

#### Monitoring Options:
1. **AWS Console**: You can check the endpoint status in the [SageMaker Endpoints section](https://us-east-1.console.aws.amazon.com/sagemaker/home?region=us-east-1#/endpoints) of the AWS Console.
2. **Programmatic Check**: Alternatively, you can use the `DescribeEndpoint` API to programmatically get the status of the endpoint. This is what the loop does at the end of the cell bellow for you.

In [ ]:
# Create the SageMaker endpoint using the specified configuration
create_sagemaker_endpoint(inference_config)

# Wait for the endpoint to be ready, fail, or time out
wait_for_endpoint(inference_config['model_endpoint_name'], sm_boto3, check_interval=30, timeout_minutes=30)

# 5. Start Inference Job

### Starting Inference

With the model endpoint now **ready**, we can proceed to run the inference process!

### Monitoring Job Progress:
- The best way to track progress is by reading the **SQS queues** used in the inference pipeline.
- Keep in mind that some images might **fail**, so the counts for "total sent" and "total processed" may not match exactly.

### Job Completion:
- The job is considered **complete** when the **'total images processed'** count does not change over a period of **5 minutes**.
- Run the next cell, and the rest below, **only when the job is complete**.
- The `wait_for_job_completion` function helps track the progress of the inference job by monitoring the number of processed GeoJSON files in the specified S3 bucket.

#### How It Works:
1. **Checks GeoJSON Results**: The function compares the number of GeoJSON files in the output S3 path to the total number of input images.
2. **Completion Criteria**: The job is considered complete when the number of processed GeoJSON files equals or exceeds the total number of input images.
3. **Timeout Handling**: 
   - The function waits for updates in a loop, checking the progress at regular intervals (`check_interval`).
   - If no updates are detected within the timeout period (`timeout_minutes`), the function exits, indicating the job did not complete in time.

In [ ]:
# Spin up some tasks for ModelRunner
update_modelrunner_tasks(inference_config, 3)

# Submit all selected images for inference processing
submit_all_images_for_processing(inference_config, all_images[:])

# Wait for the job to complete and report the status
wait_for_job_completion(inference_config, all_images)

# 6. Stop Model Endpoint

### Important: Cost of the Model Endpoint

The model endpoint is the **most expensive component** of the pipeline. **Do not leave it running** when not in use!

#### Cost Breakdown:
- **G4dn.2xlarge**: ~$0.75/hour
- **P3.2xlarge**: ~$3.00/hour  
[View AWS EC2 On-Demand Pricing](https://aws.amazon.com/ec2/pricing/on-demand/)

**Note:** All benchmarking has been conducted using **P3 instances** for optimal performance.

In [ ]:
# Reset the number of model runner tasks to zero
update_modelrunner_tasks(inference_config, 0)
print("✅ Model runner tasks have been reset to zero.")

# Attempt to delete the SageMaker endpoint
delete_sagemaker_endpoint(inference_config['model_endpoint_name'], sm_session)


# 7. Post Processing

### Download and Explore Your Results

After running the inference pipeline, you can:
- **Download the Shapefile or GeoJSON**: Use these outputs with your favorite workflow tools, such as **ArcGIS**, **QGIS**, or [**Kepler.gl**](https://kepler.gl/).
- **Preview the Results in the Notebook**: Visualize individual images with their corresponding GeoJSON footprints to ensure the results meet your expectations.

In [ ]:
# Downloading Your Results

# Each 10k tile has a geojson result that needs to be merged into a single geojson.
merge_and_save_results_local(inference_config)

### Perform Post-Processing

If the resulting features require further refinement (e.g., correcting rounded corners), refer to the **`doc/orthogonalization`** guide for examples of post-processing techniques.

In [ ]:
# Visualizing Results for a Single Image

# Note: Visualizing the entire AOI is not practical in this notebook.
# Instead, we can view any of the individual images (e.g., one of the 10k images),
# overlay with the resulting GeoJSON footprints.
# The output S3 bucket contains GeoJSON results with the same filenames as the images.

# Specify the index of the image to view
image_to_view = 0  # Change this index to visualize a different image

# Select the image and corresponding GeoJSON
image = all_images[image_to_view]
geojson = get_geojson_name_for(inference_config, image)

# Set the detection threshold
# Option 1: Use the threshold from the configuration file
thresh = inference_config['detection_threshold']

# Option 2: Override the threshold value here
# Uncomment the next line to override
# thresh = 0.3

# Display the threshold being used
print(f"Using detection threshold: {thresh}")

# Plot the image with the GeoJSON footprints (this may take a few moments)
plot_inference_results(image, geojson, thresh)

## Comparing Inference Results

The `compare_inference_results` function allows you to visually compare the outputs of two models for a given input image. This is particularly useful for evaluating improvements or differences between models.

### Usage Example:
To compare results, call the function with:
1. The input image file path (local or S3).
2. The S3 URI for the first model's GeoJSON output.
3. The S3 URI for the second model's GeoJSON output.

In [ ]:
compare_inference_results(
    "s3://osml-test-images-12345678901/small.tif",
    "s3://mr-bucket-sink-12345678901/new_model_job_id/new_model_output.geojson",
    "s3://mr-bucket-sink-12345678901/old_model_job_id/old_model_output.geojson"
)

# 8. Filter and Save Detections

### Save Filtered Results Using a Threshold

Once you have identified an appropriate detection threshold, you can apply it to filter the results. This process involves:

1. **Filtering Features**: The threshold is used to retain only the features that meet your confidence level, ensuring the output focuses on the most reliable detections.
2. **Saving Results**:
   - A filtered **GeoJSON file** is created, containing only the features that pass the threshold.
   - The GeoJSON can also be saved as a **Shapefile** for compatibility with a wider range of tools.

These filtered outputs can be used for further analysis, visualization, or integration into workflows with tools like ArcGIS, QGIS, or Kepler.gl.

In [ ]:
# Set the detection threshold
# Option 1: Use the threshold from the configuration file
thresh = inference_config['detection_threshold']

# Option 2: Override the threshold value here
# Uncomment the next line to override
# thresh = 0.3

# Display the threshold being used
print(f"Using detection threshold: {thresh}")

# Step 1: Save a new GeoJSON file with the filtered results
thresholded_results_filename = filter_geojson_with_threshold(inference_config, thresh)

# Step 2: Convert and save the filtered GeoJSON as a Shapefile
save_geojson_as_shapefile(inference_config, thresh, thresholded_results_filename)